In [1]:
import pandas as pd
import numpy as np
from numpy import array

import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Binarizer,OneHotEncoder
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import scipy
from scipy import stats,interpolate
from scipy.stats import skew,kurtosis,describe,variation,shapiro,levene,f_oneway
from scipy.special import kl_div
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
def describex(data):
        data = pd.DataFrame(data)
        stats = data.describe()
        skewness = data.skew()
        kurtosis = data.kurtosis()
        skewness_df = pd.DataFrame({'skewness':skewness}).T
        kurtosis_df = pd.DataFrame({'kurtosis':kurtosis}).T
        Appended_Stats = pd.concat([stats,skewness_df,kurtosis_df],axis=0)
        return Appended_Stats

In [3]:
data1 = pd.read_csv('train_input_스핀들전력.csv',encoding='cp949')
data2 = pd.read_csv('train_input_스핀들진동.csv',encoding='cp949')
data3 = pd.read_csv('train_input_전체전력.csv',encoding='cp949')
data4 = pd.read_csv('train_output.csv',encoding='cp949')

In [4]:
for col in data1.columns:
    data1.rename(columns={col:col.replace("s"," ")},inplace=True)
data1.columns = 'SpindlePower_' + data1.columns.astype(str)

for col in data2.columns:
    data2.rename(columns={col:col.replace("s"," ")},inplace=True)
data2.columns = 'SpindleVibration_' + data2.columns.astype(str)

for col in data3.columns:
    data3.rename(columns={col:col.replace("s"," ")},inplace=True)
data3.columns = 'SpindleComprehensive_' + data3.columns.astype(str)

In [5]:
data1.head()

,SpindlePower_0,SpindlePower_1,SpindlePower_2,SpindlePower_3,SpindlePower_4,SpindlePower_5,SpindlePower_6,SpindlePower_7,SpindlePower_8,SpindlePower_9,...,SpindlePower_50,SpindlePower_51,SpindlePower_52,SpindlePower_53,SpindlePower_54,SpindlePower_55,SpindlePower_56,SpindlePower_57,SpindlePower_58,SpindlePower_59
0,0.000000,0.687045,9.515879,0.400000,0.866667,0.041073,0.000000,0.000000,0.000000,0.814193,...,8.964927,2.860822,5.930431,1.323406,0.822217,1.511922,0.000000,10.903064,19.651940,5.794539
1,87.627847,60.546384,159.635294,57.760371,95.066726,160.938912,22.621917,84.337811,118.616418,152.895025,...,29.765552,71.998994,123.709176,24.569416,147.386684,139.555232,24.961226,170.976344,49.789691,85.663291
2,4.590282,5.448428,22.306119,1.725718,0.000000,2.672039,7.356061,0.577096,12.351741,0.974032,...,5.952429,7.372741,0.555185,0.000000,9.470237,12.557354,2.490987,4.821944,8.547896,8.847617
3,3.612226,0.000000,18.318390,20.145843,3.657803,0.000000,2.266934,0.525176,3.938351,0.908532,...,9.741470,6.737982,11.546517,2.398036,0.000000,0.000000,13.784338,9.728492,4.999573,0.796579
4,66.666052,69.907073,48.902432,64.610148,69.245092,133.616582,64.689094,144.457408,78.087866,52.016541,...,14.815025,51.856295,109.292488,48.483833,45.098933,72.499428,78.904331,51.392993,52.770880,63.026120


In [6]:
data2.head()

,SpindleVibration_0,SpindleVibration_1,SpindleVibration_2,SpindleVibration_3,SpindleVibration_4,SpindleVibration_5,SpindleVibration_6,SpindleVibration_7,SpindleVibration_8,SpindleVibration_9,...,SpindleVibration_50,SpindleVibration_51,SpindleVibration_52,SpindleVibration_53,SpindleVibration_54,SpindleVibration_55,SpindleVibration_56,SpindleVibration_57,SpindleVibration_58,SpindleVibration_59
0,0.050412,0.059542,0.050460,0.050000,0.059524,0.050451,0.059569,0.040898,0.059100,0.040843,...,0.062927,0.057081,0.062859,0.057151,0.062942,0.057022,0.062900,0.057129,0.066424,0.060000
1,0.050749,0.033049,0.045882,0.060000,0.053935,0.045894,0.050802,0.031277,0.035423,0.039569,...,0.027425,0.036167,0.056270,0.027465,0.036228,0.055763,0.035839,0.038126,0.048154,0.041889
2,0.051423,0.040874,0.040000,0.040000,0.040000,0.040000,0.040000,0.040000,0.040000,0.040000,...,0.051685,0.040813,0.040000,0.040000,0.040000,0.047878,0.040928,0.040000,0.049269,0.040794
3,0.050000,0.050000,0.041898,0.048133,0.041868,0.040000,0.040000,0.040000,0.040000,0.040000,...,0.051853,0.050000,0.041891,0.040000,0.040000,0.040000,0.040000,0.040000,0.040000,0.040000
4,0.057846,0.042106,0.058919,0.048992,0.039696,0.030000,0.030000,0.030000,0.030000,0.030000,...,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000


In [7]:
data3.head()

,SpindleComprehensive_0,SpindleComprehensive_1,SpindleComprehensive_2,SpindleComprehensive_3,SpindleComprehensive_4,SpindleComprehensive_5,SpindleComprehensive_6,SpindleComprehensive_7,SpindleComprehensive_8,SpindleComprehensive_9,...,SpindleComprehensive_50,SpindleComprehensive_51,SpindleComprehensive_52,SpindleComprehensive_53,SpindleComprehensive_54,SpindleComprehensive_55,SpindleComprehensive_56,SpindleComprehensive_57,SpindleComprehensive_58,SpindleComprehensive_59
0,2280.204292,2273.543763,2280.336096,2278.418432,2284.741078,2301.038542,2320.032293,2312.039056,2326.817772,2260.593195,...,2280.910731,2272.186802,2279.686973,2267.788800,2274.577912,2271.342258,2282.162620,2300.722765,2328.953463,2334.204294
1,2837.098017,2390.058312,2541.186453,2993.064531,2760.500000,2665.568623,2896.593024,2302.333878,2519.502672,2896.287441,...,2610.933590,2772.194148,2468.630097,2602.013184,2764.299209,2536.141681,2606.330055,2744.927473,2758.144704,2617.634316
2,2371.324729,2370.161076,2377.836154,2491.133333,2352.784600,2194.035999,2192.908176,2195.623139,2191.399802,2189.930151,...,2308.733865,2362.321729,2328.495625,2518.575964,2539.887556,2309.608472,2186.789773,2185.043114,2187.560079,2192.697872
3,2362.850888,2288.887064,2373.969992,2319.877160,2486.628566,2171.574229,2174.168297,2170.991084,2168.789810,2172.744587,...,2335.852659,2222.432800,2358.414458,2372.075568,2333.112703,2134.428782,2167.424849,2170.110323,2162.033899,2169.040949
4,2335.002235,2566.642514,2475.106175,2365.970656,2782.710670,2216.492950,2687.474753,2631.922709,2309.048228,2231.026368,...,2489.491977,2352.754638,2758.202634,2105.135265,2157.721111,2174.143429,2180.165406,2159.479879,2164.937126,2160.647059


In [8]:
data4.head() # Tool --> 공구종류 / condition --> 공구상태 {0:정상 1:불량}

,Tool,condition
0,0,1
1,2,0
2,0,1
3,0,0
4,1,1


In [9]:
data_full = pd.concat([data1,data2,data3,data4],axis=1)

In [10]:
data_full.shape

(410, 182)

In [11]:
Data = data_full
Target = 'condition'
Input_Data = data_full.drop(['Tool','condition'],axis=1)

In [12]:
def Analysis(Data,Target,Input_Data):
    
    Stats = describex(Input_Data)
    
    print('Class Count','\n',Data[Target].value_counts(),'\n')
    print('Normal', round(Data[Target].value_counts()[0]/len(Data) * 100,2), '% of the dataset')
    print('Fault', round(Data[Target].value_counts()[1]/len(Data) * 100,2), '% of the dataset')
    print('Imbalanced Ratio(IR):',round(Data[Target].value_counts()[0]/Data[Target].value_counts()[1]*1,2),'%')
    
    sns.set(font_scale=1.0)
    plt.figure(figsize=(3.5,3))
    plt.grid(True)
    plt.title('Machine \n (0: Normal || 1: Fault)', fontsize=11)
    colors = ["blue", "red"]
    sns.countplot(x= Target, data=Data,palette=colors)
    print('\n')
    
    for col in Data.columns:
        
        try : Data[col] = Data[col].astype('float')
        except: pass  
    
    fig, ax = plt.subplots(61,3, figsize=(30, 400)) # 데이터 개수에 따라 플롯 형태 변경 가능 
    fig.subplots_adjust(hspace = .2, wspace=.3, )
    ax = ax.ravel()

    for i in range(len(Input_Data.columns)):
        ax[i] = Input_Data.iloc[:,i].plot(kind='hist', ax=ax[i],color='blue',alpha=0.5,secondary_y=True,bins=5)
        plt.title(Input_Data.columns[i])
        
    print('\n')    
    df_fault = Data[Data[Target]==1] #.set_index('TimeStamp')
    df_features = Data[Input_Data.columns] #.set_index('TimeStamp')

    ColumnNames = df_features.columns

    for feature in ColumnNames:
        sns.set_context('talk')
        _ = plt.figure(figsize=(50,8))
        _ = plt.plot(df_fault[feature], linestyle='none', marker='X', color='red', markersize=6)
        _ = plt.plot(df_features[feature], color='grey')
        _ = plt.title(feature)
        plt.show()    
        
    print('\n')    
    df_normal =  Data[Data[Target]==0][Input_Data.columns]
    df_fault = Data[Data[Target]==1][Input_Data.columns]
    
    Equal_VarTest = pd.DataFrame(stats.f_oneway(df_normal[Input_Data.columns],df_fault[Input_Data.columns])[1]).rename(columns={0:'Variance Equality F-Test '}).round(3)   
    Columns = pd.DataFrame(Data[Input_Data.columns].columns).rename(columns = {0:'Variables'})
    
    T_Statistic = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=True)[0].round(3)).rename(columns={0:'T-Statistic'})
    P_value = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=True)[1].round(3)).rename(columns={0:'P-Value'})
    df_significance = pd.concat([Columns,T_Statistic,P_value],axis=1) # 등분산성 가정 충족  
    
    T_Statistic = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=False)[0].round(3)).rename(columns={0:'T-Statistic-Welch'})
    P_value = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=False)[1].round(3)).rename(columns={0:'P-Value-Welch'})
    df_significance_Welch = pd.concat([T_Statistic,P_value],axis=1) # 등분산성 가정 위배 
    
    #print('\n')
    #plt.rcParams["figure.figsize"] = (125,100)
    #plt.title('Correlation Matrix',fontsize=25) 
    #sns.set(font_scale=1.0)
    #sns.heatmap(data = pd.concat([Data[Target],Input_Data],axis=1).corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='viridis_r')
    #plt.xticks(rotation=90) 
    
    df_corr = pd.concat([Data[Target],Input_Data],axis=1).corr().iloc[1:,0]
    df_corr = pd.DataFrame(df_corr).rename(columns={Target:'Corr'}).reset_index()['Corr']
    print('\n')
    
    X = Input_Data
    scaler = StandardScaler()
    scale_cols =  X.columns
    scaled_X = scaler.fit_transform(X[scale_cols])
    scaled_X = pd.DataFrame(scaled_X, columns=scale_cols)
    scaled_X = scaled_X.set_index(X.index)

    df_vif = pd.DataFrame()
    df_vif['VIF_Factor'] = [variance_inflation_factor(scaled_X.values, i) 
                         for i in range(scaled_X.shape[1])]
    
    Variable_Explain = pd.concat([df_significance,df_significance_Welch,Equal_VarTest,df_corr,df_vif],axis=1)
    
    print('Normality Test'.center(80))
    print('\n')
    
    for i in range(len(Input_Data.columns)):
        
        print(Input_Data.columns[i],stats.shapiro(Input_Data[Input_Data.columns[i]]),'\n')
        
    
    return  Stats, Variable_Explain #, LR_Coeff

In [13]:
#Stats, Variable_Explain = Analysis(Data,Target,Input_Data)

In [14]:
#Stats

In [15]:
#Variable_Explain

In [16]:
#Variable_Explain.sort_values(by='P-Value',ascending=True)

In [17]:
#Variable_Explain.sort_values(by='P-Value-Welch',ascending=True)

In [18]:
#Variable_Explain.sort_values(by='Corr',ascending=False)

In [19]:
#Variable_Explain.sort_values(by='VIF_Factor',ascending=True)